In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
from github import Github, GithubException
import time
from shutil import copyfile
from pymongo import MongoClient
import re
from bson.objectid import ObjectId
import pprint

In [2]:
client = MongoClient()
db = client['repos-database']
dockers = db['docker_repos']

In [47]:
cursor = dockers.find()
# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))

In [84]:
linguagens = []
dicLinguagens = []
count = 0
for i in df.itertuples():
    lang = i.Linguagens
    if (lang == None):
        count = count + 1
        continue
    else:
        lang = lang[0].split(" ")[0].lower()
        if(lang in linguagens):
            index = linguagens.index(lang)
            dicLinguagens[index]['qtd'] = dicLinguagens[index]['qtd'] + 1
        else:
            linguagens.append(lang)
            dicLinguagens.append({"nome":lang, "qtd": 1}) 

In [94]:
dataLinguagens = pd.DataFrame(dicLinguagens,columns=['nome','qtd'])
#dataLinguagens.sort_values('qtd',ascending=False)